Ejemplo tomado de:
    https://towardsdatascience.com/genetic-algorithms-in-python-using-the-deap-library-e67f7ce4024c

Un ejemplo de como aplicar la optimizacion con AGs

Cuando se esta optimizando un plan de alimentacion, hay muchsa cosa que se deben tener en cuenta. En este ejemplo, se va hacer un plan de semanal de alimentacion paar una persona que ya decidio cuantas calorias desea consumir y que porcentaje debe llagar de proteinas, carbohidratos y gordos.

Ademas de estos valore fijos, el programa de optimizacion tambien necesita una lista predefinida de productos para escoger. Para cada uno de esos productos, sabemos el numero de calorias por unidad, asi como tambien los porcentajes de gordo, proteinas y carbohidratos.


Las metas de cantidades para la persona de nuestro ejemplo son:
    
Total calorias= 2500 * 7 dias = 17500 calorias
Percentaje Proteina = 30%
Percentaje Carbohidratos = 50%
Percentaje Gordo = 20%

La lista de los productos de los cuales se puede escoger son:

![Alimentos.JPG](attachment:Alimentos.JPG)

La meta de la optimizacion

La meta del programa de optimizacion es encontrar una lista de productos para la semana, de los cuales las calorias y los macronutrientes del total semanal sea tan cercano como sea posible a las cantidades de la meta.

Genetic Algorithms for optimization

Many optimization algorithms exist and Genetic Algorithms are one of them. Genetic Algorithms are fascinating because the idea behind them is very intriguing: Genetic Algorithms are based on the idea of natural selection in genetics.

Applying Genetic Algorithms to other optimizations

For other use cases of optimization, Genetic Algorithms implement the same approach of iterative mating, mutation, and selection. Rather than using a sequence of genes as an individual, it can be many other things.

In the example of optimizing meal plan, for example, we can write the “shopping list” as an individual. Buying 1000 bananas and nothing else would be a relatively bad shopping list. Buying 10 of every item would be much more realistic already.

Genetic Algorithms in Python

Sounds amazing, let’s do a practical case in Python. We will use the DEAP library, because it has a lot of genetic functions that we can reuse. You can follow along with the total notebook over here.
Set up inputs in Python

First, let’s fix the goal nutriment quantities. I’ve set them in different ways because in practice they are often expressed in percentage per nutriment, but for the optimization, it’s easier to have them in grams.

In [2]:
pip install deap

In [3]:
import pandas as pd
import numpy as np
import random
from deap import base
from deap import creator
from deap import tools

In [4]:
#porcentajes meta
total_calories = 2500 * 7
percentage_prot = 0.3
percentage_carb = 0.5
percentage_fat = 0.2

# calcula el total de calorias por macronutriente
cal_prot = round(percentage_prot * total_calories)
cal_carb = round(percentage_carb * total_calories)
cal_fat = round(percentage_fat * total_calories)
print("calorias de proteina: ",cal_prot,"calorias de carb: ",cal_carb,"calorias de grasa: ",cal_fat)

# fija la informacion sobre macronutriments: calorias por gramo de proteina, carb and grasa
prot_cal_p_gram = 4
carb_cal_p_gram = 4
fat_cal_p_gram = 9

#gramos de la meta
gram_prot = cal_prot / prot_cal_p_gram
gram_carb = cal_carb / carb_cal_p_gram
gram_fat = cal_fat / fat_cal_p_gram
print("gramos de proteina: ", gram_prot,"gramos de carb: ",gram_carb,"gramos de grasa: ",gram_fat)

calorias de proteina:  5250 calorias de carb:  8750 calorias de grasa:  3500
gramos de proteina:  1312.5 gramos de carb:  2187.5 gramos de grasa:  388.8888888888889


In [5]:
# por semana: min, max, cal/unidad, g de prot, g de grasa, g de carb
products_table = pd.DataFrame.from_records([
    ['Banano 1u', 0, 4, 89, 1, 0, 23],
    ['Mandarina 1u', 0, 4, 40, 1, 0, 10],
    ['Piña 100g', 0, 7, 50, 1, 0, 13],
    ['Uvas 100g', 0, 7, 76, 1, 0, 17],
    ['Chocolate 1 bar', 0, 4, 230, 3, 13, 25],

    ['Queso Paipa 100g', 0, 8, 350, 28, 26, 2],
    ['Quesillo 100g', 0, 8, 374, 18, 33, 1],
    ['Pesto 100g', 0, 8, 303, 3, 30, 4],
    ['Hummus 100g', 0, 8, 306, 7, 25, 11],
    ['Pasta de berenjena 100g', 0, 4, 228, 1, 20, 8],

    ['Batido de proteinas', 0, 5, 160, 30, 3, 5],
    ['Hamburguesa vegetariana 1', 0, 5, 220, 21, 12, 3],
    ['Hamburguesa vegetariana 2', 0, 12, 165, 16, 9, 2],
    ['Huevo cocido 1', 0, 8, 155, 13, 11, 1],
    ['Huevo frito 1', 0, 16, 196, 14, 15, 1],

    ['Medio baguette', 0, 3, 274, 10, 0, 52],
    ['Pan tajado 1 tajada', 0, 3, 97, 3, 1, 17],
    ['Pizza de queso1u', 0, 3, 903, 36, 47, 81],
    ['Pizza vegetariana 1u', 0, 3, 766, 26, 35, 85],

    ['Leche de soya 200ml', 0, 1, 115, 8, 4, 11],
    ['Leche de soya achocolatada 250ml', 0, 3, 160, 7, 6,20],

])
products_table.columns = ['Nombre', 'Min', 'Max', 'Calorias', 'Gram_Prot', 'Gram_Grasa', 'Gram_Carb']

products_table

,Nombre,Min,Max,Calorias,Gram_Prot,Gram_Grasa,Gram_Carb
0,Banano 1u,0,4,89,1,0,23
1,Mandarina 1u,0,4,40,1,0,10
2,Piña 100g,0,7,50,1,0,13
3,Uvas 100g,0,7,76,1,0,17
4,Chocolate 1 bar,0,4,230,3,13,25
5,Queso Paipa 100g,0,8,350,28,26,2
6,Quesillo 100g,0,8,374,18,33,1
7,Pesto 100g,0,8,303,3,30,4
8,Hummus 100g,0,8,306,7,25,11
9,Pasta de berenjena 100g,0,4,228,1,20,8


In [6]:
# Agregar columna de costos (ejemplo, puedes ajustar los valores según precios reales)
products_table['Costo'] = [
    0.5, 0.4, 0.7, 0.8, 1.5,   # frutas y chocolate
    3.5, 3.0, 2.5, 2.8, 2.0,   # quesos, hummus, etc.
    2.5, 4.0, 3.5, 0.5, 0.8,   # proteicos
    1.0, 0.6, 6.0, 5.5,        # panes y pizzas
    1.2, 1.5                   # leches
]


In [7]:
cost_data = list(products_table['Costo'])

In [8]:
# extrae la informacion de los products en un format mas facil
# para usar en la funcion de costo de los algoritmos deap
cal_data = products_table[['Gram_Prot', 'Gram_Grasa', 'Gram_Carb']]

prot_data = list(cal_data['Gram_Prot'])
fat_data = list(cal_data['Gram_Grasa'])
carb_data = list(cal_data['Gram_Carb'])

In [9]:
# Aqui se hace la iniciacion aleatoria del AG
# da una lista de enteros con el numero de unidades para cada producto
def n_per_product():
    return random.choices( range(0, 10), k = 21)

In [10]:
# Esta es la funcion de aptitud para la evalucion de cada lista, o individuo
# Se tomo como la diferencia absoluta entre el numero de calorias planeadas y la meta de calorias
def evaluate(individual):
    individual = individual[0]
    tot_prot = sum(x*y for x,y in zip(prot_data,individual))
    tot_fat = sum(x*y for x,y in zip(fat_data,individual))
    tot_carb = sum(x*y for x,y in zip(carb_data,individual))
    cals = prot_cal_p_gram * tot_prot + carb_cal_p_gram * tot_carb + fat_cal_p_gram * tot_fat

    # calcular diferencia de calorías
    cal_diff = abs(cals - total_calories)

    # calcular costo total
    total_cost = sum(x*y for x,y in zip(cost_data, individual))

    return cal_diff, total_cost


#def evaluate(individual):
 #   individual = individual[0]
  #  tot_prot = sum(x*y for x,y in zip(prot_data,individual))
   # tot_fat = sum(x*y for x,y in zip(fat_data,individual))
  #  tot_carb = sum(x*y for x,y in zip(carb_data,individual))
   # cals = prot_cal_p_gram * tot_prot + carb_cal_p_gram * tot_carb + fat_cal_p_gram * tot_fat
    #return abs(cals - total_calories),

In [11]:
# Aqui se conforma el algoritmo con la libreria deap:
# se registran las diferentes funciones en el toolbox

#creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
#creator.create("Individual", list, fitness=creator.FitnessMin)
creator.create("FitnessMulti", base.Fitness, weights=(-1.0, -1.0))
creator.create("Individual", list, fitness=creator.FitnessMulti)

toolbox = base.Toolbox()

toolbox.register("n_per_product", n_per_product)

toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.n_per_product, n=1)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

toolbox.register("evaluate", evaluate)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)
toolbox.register("select", tools.selNSGA2)

#toolbox.register("select", tools.selTournament, tournsize=3)

In [12]:
toolbox.population(n=10)

[[[9, 5, 8, 1, 4, 9, 6, 4, 1, 3, 0, 7, 8, 9, 6, 4, 0, 4, 9, 0, 5]],
 [[9, 9, 0, 7, 7, 3, 6, 9, 8, 5, 8, 0, 1, 3, 7, 3, 2, 5, 5, 8, 0]],
 [[3, 6, 4, 6, 0, 8, 3, 9, 5, 7, 7, 2, 0, 6, 9, 7, 9, 1, 5, 8, 3]],
 [[9, 1, 0, 8, 9, 8, 6, 7, 4, 5, 5, 9, 7, 2, 9, 5, 1, 0, 1, 0, 1]],
 [[1, 6, 6, 5, 2, 5, 4, 5, 4, 9, 7, 4, 5, 6, 4, 3, 9, 6, 3, 0, 4]],
 [[2, 3, 7, 0, 8, 2, 3, 3, 4, 2, 7, 6, 7, 7, 4, 9, 6, 4, 7, 4, 0]],
 [[1, 5, 2, 6, 3, 5, 3, 6, 8, 5, 4, 3, 9, 8, 3, 4, 1, 5, 1, 7, 1]],
 [[0, 8, 8, 5, 9, 8, 8, 7, 2, 4, 1, 9, 7, 9, 8, 6, 6, 8, 7, 3, 9]],
 [[2, 9, 7, 6, 2, 8, 4, 3, 9, 1, 0, 7, 6, 2, 2, 3, 0, 7, 2, 1, 1]],
 [[5, 1, 6, 1, 7, 7, 6, 8, 5, 2, 5, 2, 9, 0, 0, 2, 6, 1, 2, 7, 5]]]

In [13]:
# Esta es la definicion de todo el algoritmo genetico
"""
def main():
    pop = toolbox.population(n=300)

    # Evalua toda la poblacion
    fitnesses = list(map(toolbox.evaluate, pop))
    for ind, fit in zip(pop, fitnesses):
        ind.fitness.values = fit

    # CXPB  es la probabilidad con la que se cruzan dos individuos
    #
    # MUTPB es la probabilidad de mutacion de un individuo
    CXPB, MUTPB = 0.5, 0.2

    # Se encuentran todas las aptitudes
    fits = [ind.fitness.values[0] for ind in pop]

    # Variable que lleva el numero de generaciones
    g = 0

    # Inicia la evolucion
    while g < 5000:
        # Una nueva generacion
        g = g + 1
        #print("-- Generacion %i --" % g)

        # Selecciona los individuos para la siguiente generacion
        offspring = toolbox.select(pop, len(pop))
        # Clona los individuos seleccionados
        offspring = list(map(toolbox.clone, offspring))

        # Aplica el cruce y la mutacion a la decendencia
        for child1, child2 in zip(offspring[::2], offspring[1::2]):
            if random.random() < CXPB:
                toolbox.mate(child1[0], child2[0])
                del child1.fitness.values
                del child2.fitness.values

        for mutant in offspring:
            if random.random() < MUTPB:
                toolbox.mutate(mutant[0])
                del mutant.fitness.values

        # Evalua los individuos con una funcion de aptitud invalida
        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        fitnesses = map(toolbox.evaluate, invalid_ind)
        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit

        pop[:] = offspring

        # Recoge todas las aptitudes en una lista e imprime los estados
        fits = [ind.fitness.values[0] for ind in pop]

        length = len(pop)
        mean = sum(fits) / length
        sum2 = sum(x*x for x in fits)
        std = abs(sum2 / length - mean**2)**0.5

        #print(min(fits), max(fits), mean, std)

    best = pop[np.argmin([toolbox.evaluate(x) for x in pop])]
    return best
"""
def main():
    pop = toolbox.population(n=300)
    fitnesses = list(map(toolbox.evaluate, pop))
    for ind, fit in zip(pop, fitnesses):
        ind.fitness.values = fit

    CXPB, MUTPB = 0.5, 0.2
    g = 0

    while g < 200:  # menos generaciones porque multi-objetivo converge más lento
        g += 1
        offspring = toolbox.select(pop, len(pop))
        offspring = list(map(toolbox.clone, offspring))

        for child1, child2 in zip(offspring[::2], offspring[1::2]):
            if random.random() < CXPB:
                toolbox.mate(child1[0], child2[0])
                del child1.fitness.values
                del child2.fitness.values

        for mutant in offspring:
            if random.random() < MUTPB:
                toolbox.mutate(mutant[0])
                del mutant.fitness.values

        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        fitnesses = map(toolbox.evaluate, invalid_ind)
        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit

        pop[:] = offspring

    return pop

pop = main()
pareto_front = tools.sortNondominated(pop, len(pop), first_front_only=True)[0]



# Resultado del AG


El mejor individuo,de la ultima generacion es al lista que se muestra abajo como la columna The best individual of the last generation is the list that is shown below as the column “escog_univariada”.

In [14]:
best_solution = pareto_front[0]  # puedes explorar varias

In [15]:

products_table['escog_univariada'] = pd.Series(best_solution[0])
products_table.head()

,Nombre,Min,Max,Calorias,Gram_Prot,Gram_Grasa,Gram_Carb,Costo,escog_univariada
0,Banano 1u,0,4,89,1,0,23,0.5,0
1,Mandarina 1u,0,4,40,1,0,10,0.4,0
2,Piña 100g,0,7,50,1,0,13,0.7,0
3,Uvas 100g,0,7,76,1,0,17,0.8,0
4,Chocolate 1 bar,0,4,230,3,13,25,1.5,0
